# What Makes a Movie Successful?

## A Deeper Dive Behind the Box Office Total

### We will be analyzing the following features to answer the above question.
1. Include only movies that were released in the United States2. Include only movies that were released 2000 - 2022 (startYear >=2000 and startYear<=2022)3. Include only full-length movies (titleType = "movie".
4. 
Exclude movies that are missing genre or runte
5. 
Include only fictional genres (where Genres does not include "Documentar"..)

In [26]:
# Forming data folder to hold relevant info
import os
os.makedirs('Data/',exist_ok=True) 

In [27]:
# Confirming folder was created and files added successfully
os.listdir("Data/")

['.ipynb_checkpoints',
 'Basics.csv.gz',
 'IMDB Movie Dataset Info.docx',
 'Ratings.csv.gz',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

In [28]:
# Files ready. Importing pandas to display data
import pandas as pd
# We want all columns displayed, setting output to 50 to be safe
pd.set_option('display.max_columns',50)

In [29]:
# Loading in data set with USA movies only
us_path = "Data/title-akas-us-only.csv"
df_us = pd.read_csv(us_path, low_memory=False)
df_us.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [30]:
# Now loading title basics
tb_path = "Data/title.basics.tsv.gz"
basics = pd.read_csv(tb_path, sep='\t', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [31]:
# Removing non-US movies from list with filter
us_filter = basics['tconst'].isin(df_us['titleId'])
basics = basics[us_filter]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


In [32]:
us_filter

0            True
1            True
2           False
3           False
4            True
            ...  
10017006    False
10017007    False
10017008    False
10017009    False
10017010    False
Name: tconst, Length: 10017011, dtype: bool

In [33]:
# We should replace placeholder \N with actual null values before preprocessing further
import numpy as np
df_us = df_us.replace({'\\N': np.nan})
df_us.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [34]:
basics = basics.replace({'\\N': np.nan})
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,NaN,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,NaN,1,"Short,Sport"


In [35]:
# Dropping rows from genres and runtime minutes columns with any nulls
# To be efficient, we'll locate just the columns as a variable
basics = basics.dropna(subset = ['runtimeMinutes','genres'], how = 'any')
# Checking column to be sure
basics['genres'].isna().sum()

0

In [36]:
basics['titleType'].value_counts()

short           278334
movie           203476
tvEpisode       173684
video           115616
tvSeries         36847
tvMovie          25949
tvSpecial         9289
tvMiniSeries      5845
tvShort           4101
videoGame          194
Name: titleType, dtype: int64

In [37]:
# We only want full length movies, so adding filter for titleType to display such
movie_filter = basics['titleType']=='movie'
basics=basics[movie_filter]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [38]:
# Since we only want fictional genres, we will exclude documentaries
docu_filter = basics['genres'].str.contains("Documentary")
basics = basics[~docu_filter]

In [51]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
10016149,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10016693,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [40]:
# Start year should be a float datatype
basics['startYear'].dtype

dtype('O')

In [41]:
basics['startYear'] = basics['startYear'].astype(float)
basics['startYear'].dtype

dtype('float64')

In [42]:
# We only want movies made from year 2000 to 2022. Filtering as such.
basics['startYear'].max()

2029.0

In [43]:
basics = basics.loc[basics['startYear'].isin(range(2000, 2023))]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
10016149,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10016693,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [44]:
basics['startYear'].max()

2022.0

In [45]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  object 
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86979 non-null  object 
 8   genres          86979 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


In [46]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [57]:
# We need to save basics dataframe as a csv in our data folder
# Need to specify index is false to prevent adding redundant column 
basics.to_csv('Data/Basics.csv', index=False)

In [47]:
# Loading title ratings data into new dataframe
ratings_path = "Data/title.ratings.tsv.gz"
ratings = pd.read_csv(ratings_path, sep='\t', low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1988
1,tt0000002,5.8,265
2,tt0000003,6.5,1849
3,tt0000004,5.5,178
4,tt0000005,6.2,2632


In [48]:
# We only want ratings from our previous dataset
filter_basics = ratings['tconst'].isin(basics['tconst'])
# Filtering new dataset
ratings = ratings[filter_basics]

In [49]:
# Replacing placeholders in new set with nulls
ratings = ratings.replace({'\\N': np.nan})
ratings.head()

,tconst,averageRating,numVotes
17961,tt0035423,6.4,87153
40764,tt0062336,6.4,175
46645,tt0069049,6.7,7754
63640,tt0088751,5.2,336
69953,tt0096056,5.6,846


In [56]:
# Storing ratings to data folder as csv
ratings.to_csv('Data/Ratings.csv', index=False)

In [58]:
# Confirming that basics, akas, and ratings are all saved in data folder
os.listdir("Data/")

['.ipynb_checkpoints',
 'Basics.csv',
 'IMDB Movie Dataset Info.docx',
 'Ratings.csv',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']